In [1]:
import cv2
import numpy as np
import os

def process_and_crop(image_file, save_path):
    print(f"Processing: {image_file}")
    gray_image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    if gray_image is None:
        print(f"Failed to load image: {image_file}")
        return

    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    _, threshold_image = cv2.threshold(blurred_image, 50, 255, cv2.THRESH_BINARY)

    shape_contours, _ = cv2.findContours(threshold_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    print(f"Contours found: {len(shape_contours)}")

    largest_shape = None
    max_shape_area = 0
    for shape in shape_contours:
        approximated_shape = cv2.approxPolyDP(shape, 0.02 * cv2.arcLength(shape, True), True)
        if len(approximated_shape) == 4 and cv2.isContourConvex(approximated_shape):
            shape_area = cv2.contourArea(approximated_shape)
            if shape_area > max_shape_area:
                max_shape_area = shape_area
                largest_shape = approximated_shape

    if largest_shape is not None:
        rect_x, rect_y, rect_width, rect_height = cv2.boundingRect(largest_shape)
        square_size = max(rect_width, rect_height)
        cropped_image = gray_image[rect_y:rect_y+square_size, rect_x:rect_x+square_size]
        cv2.imwrite(save_path, cropped_image)
        print(f"Saved cropped image to: {save_path}")
    else:
        print("No square-like Petri dish found.")

# List of input image names
image_files = [
    "045_43-17-ROOT1-2023-08-08_control_pH7_-Fe+B_col0_02-Fish Eye Corrected.png",
    "051_43-11-ROOT1-2023-08-08_control_pH7_-Fe+B_col0_04-Fish Eye Corrected.png",
    "train_Dean_226324_im3.png",
    "train_Dean_230632_im5.png"
]

# Define the new output directory
output_directory = "output_cropped_images"
os.makedirs(output_directory, exist_ok=True)

# Process each image and save cropped output
for file_name in image_files:
    input_file_path = os.path.join(".", file_name)  # Assuming images are in the same folder
    output_file_path = os.path.join(output_directory, f"cropped_{file_name}")
    process_and_crop(input_file_path, output_file_path)


Processing: .\045_43-17-ROOT1-2023-08-08_control_pH7_-Fe+B_col0_02-Fish Eye Corrected.png
Contours found: 12
Saved cropped image to: output_cropped_images\cropped_045_43-17-ROOT1-2023-08-08_control_pH7_-Fe+B_col0_02-Fish Eye Corrected.png
Processing: .\051_43-11-ROOT1-2023-08-08_control_pH7_-Fe+B_col0_04-Fish Eye Corrected.png
Contours found: 13
Saved cropped image to: output_cropped_images\cropped_051_43-11-ROOT1-2023-08-08_control_pH7_-Fe+B_col0_04-Fish Eye Corrected.png
Processing: .\train_Dean_226324_im3.png
Contours found: 80
Saved cropped image to: output_cropped_images\cropped_train_Dean_226324_im3.png
Processing: .\train_Dean_230632_im5.png
Contours found: 105
Saved cropped image to: output_cropped_images\cropped_train_Dean_230632_im5.png
